In [ ]:
%load_ext autoreload
%autoreload 2

import pygplates
import numpy as np
import pygmt
import xarray as xr
from gprm.datasets import Reconstructions
from gprm import ReconstructionModel, PointDistributionOnSphere
from gprm.datasets import Seafloor
from gprm.utils.create_gpml import gpml2gdf
import seaborn as sns

import sys
sys.path.append('../degenerative_art/')
from map_effects import polygons_buffer, raster_buffer, reconstruct_and_rasterize_polygons

In [ ]:
M2016 = Reconstructions.fetch_Matthews2016()
M2021 = Reconstructions.fetch_Merdith2021()
M2019 = Reconstructions.fetch_Muller2019()
C2020 = Reconstructions.fetch_Clennett()


perspective = [195, 35]


In [ ]:

region='d'

#reconstruction_time = 600.
#reconstructed_continents = reconstruction_model.polygon_snapshot('continents', reconstruction_time)

#mask = reconstruct_and_rasterize_polygons(reconstruction_model.continent_polygons[0], reconstruction_model.rotation_model, reconstruction_time)
#prox_land, prox_ocean = raster_buffer(mask, inside='both')


def continental_drift(fig, reconstruction_model, reconstruction_time,
                      region='d', projection='Q0/37.5/10c'):

    #region='d'
    #projection='Q120/37.5/10c'
    reconstructed_continents = reconstruction_model.polygon_snapshot('continents', reconstruction_time)

    mask = reconstruct_and_rasterize_polygons(reconstruction_model.continent_polygons[0], reconstruction_model.rotation_model, reconstruction_time)
    prox_land, prox_ocean = raster_buffer(mask, inside='both')

    fig.basemap(region=region, projection=projection, frame='afg', perspective=perspective)

    prox_land.to_netcdf('./tmp.nc')

    pygmt.makecpt(cmap='gray', series=[0,800000,50000])
    fig.grdview(grid='tmp.nc', cmap=True, region=region, projection=projection, perspective=perspective, zsize='0.1c', surftype='s')

    reconstructed_continents.plot(fig, color='darkkhaki', perspective=perspective)

    fig.text(x=0.16,y=0.95,text='Continental Drift',
                 region='0/1/0/1', projection='x5c', font='14p', no_clip=True)
    fig.text(x=0.0,y=0.12,text='{:0.0f} Ma'.format(reconstruction_time),
                 region='0/1/0/1', projection='x5c', font='14p', no_clip=True)

    fig.basemap(region=region, projection=projection, frame='afg', perspective=perspective)


fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")
continental_drift(fig, M2021, reconstruction_time=600, projection='Q120/37.5/10c')
fig.savefig('./figures/continental_drift.pdf')
fig.show(width=800)


In [ ]:

def topological_model(fig, reconstruction_model, reconstruction_time,
                      region='d', projection='Q0/37.5/10c'):
    
    reconstructed_continents = reconstruction_model.polygon_snapshot('continents', reconstruction_time)
    reconstructed_plates = reconstruction_model.plate_snapshot(reconstruction_time)

    velocity_domain = PointDistributionOnSphere(distribution_type='healpix', N=8)
    velocity_field = reconstructed_plates.velocity_field(velocity_domain_features=[velocity_domain.meshnode_feature])

    fig.basemap(region='d', projection=projection, frame=['afg','+ggray90'], perspective=perspective)
    
    reconstructed_continents.plot(fig, color='gray60', perspective=perspective, region='d', projection=projection)
    reconstructed_plates.plot_subduction_zones(fig, pen='0.75p,black', gap=8, size=4, perspective=perspective, region='d', projection=projection, label='Subduction Zones')
    reconstructed_plates.plot_mid_ocean_ridges(fig, pen='0.75p,red', perspective=perspective, region='d', projection=projection, label='Mid-Ocean Ridges')
    reconstructed_plates.plot_other_boundaries(fig, pen='0.75p,gray', perspective=perspective, region='d', projection=projection)
    velocity_field.plot(fig, style='V0.1c+e', scaling=800., pen="0.5p,black", color="dodgerblue", 
                        transparency=20, perspective=perspective, region='d', projection=projection)

    colors = sns.color_palette(None, reconstructed_plates.plate_count)   
    pygmt.makecpt(cmap=','.join(['{:d}/{:d}/{:d}'.format(int(color[0]*255),int(color[1]*255),int(color[2]*255)) for color in colors]), 
                  series=[0,reconstructed_plates.plate_count,1], categorical=True)
    reconstructed_plates.plot_polygons(fig, color='+z', cmap=True, 
                                       aspatial='Z=PLATEID1', close=True,
                                       transparency=70, perspective=perspective, region=region, projection=projection)
    
    fig.legend(transparency=20, position='JBR+jBR+o-1.5c/-0.4c', box='+gwhite+p0.5p')

    fig.text(x=0.16,y=0.95,text='Plate Tectonics',
                 region='0/1/0/1', projection='x5c', font='14p', no_clip=True)
    fig.text(x=0.0,y=0.12,text='{:0.0f} Ma'.format(reconstruction_time),
                 region='0/1/0/1', projection='x5c', font='14p', no_clip=True)



fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")
topological_model(fig, M2016, reconstruction_time=330)
fig.savefig('./figures/plate_tectonics.pdf')
fig.show(width=800)


In [ ]:
import helper_functions as hf


initial_time = 0.
oldest_time = 35.
youngest_time = 0.
#final_time = 0.
reconstruction_time = 35.


def deforming_model(fig, reconstruction_model, reconstruction_time,
                    region='d', projection='Q0/37.5/10c'):
    
    tm = hf.get_topological_model(reconstruction_model)
    
    time_spans = hf.get_time_spans(tm, initial_time, oldest_time, youngest_time)

    crustal_thickness_delta = hf.get_deltas(time_spans, reconstruction_time, delta_time=-1)

    reconstructed_continents = reconstruction_model.polygon_snapshot('continents', reconstruction_time)
    reconstructed_plates = reconstruction_model.plate_snapshot(reconstruction_time)

    velocity_domain = PointDistributionOnSphere(distribution_type='healpix', N=8)
    velocity_field = reconstructed_plates.velocity_field(velocity_domain_features=[velocity_domain.meshnode_feature])


    fig.basemap(region=region, projection=projection, frame='afg', perspective=perspective)

    reconstructed_continents.plot(fig, color='gray70', pen='gray70', 
                                  region=region, perspective=perspective, transparency=60)

    pygmt.makecpt(cmap='polar', series=[-1000,1000], background='o')
    fig.grdview(crustal_thickness_delta, 
                transparency=10, perspective=perspective, region=region, projection=projection, cmap=True,
                zsize="0.1c",
                surftype="s")

    velocity_field.plot(fig, scaling=400., pen="0.1p,black", color="dodgerblue", 
                        style='V0.1c+e', transparency=30, perspective=perspective)
    reconstructed_plates.plot_deformation_zones(fig, color='-', perspective=perspective)
    reconstructed_plates.plot_subduction_zones(fig, perspective=perspective)
    reconstructed_plates.plot_mid_ocean_ridges(fig, perspective=perspective)
    reconstructed_plates.plot_other_boundaries(fig, perspective=perspective)
    
    fig.text(x=0.2,y=0.95,text='Deforming Plates',
                 region='0/1/0/1', projection='x5c', font='14p', no_clip=True)
    fig.text(x=0.0,y=0.12,text='{:0.0f} Ma'.format(reconstruction_time),
                 region='0/1/0/1', projection='x5c', font='14p', no_clip=True)
    
    with pygmt.config(FONT_ANNOT_PRIMARY='1p,white', FONT_LABEL='10p', MAP_TICK_LENGTH_PRIMARY='2p', MAP_FRAME_PEN='1p,black'):
        fig.colorbar(position='JBR+jBR+o-6.0c/0.25c+w3.5c/0.33c+h', frame=['+n','xa1000+lContraction             Extension '])


fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")
deforming_model(fig, M2019, reconstruction_time)
fig.savefig('./figures/deforming_plates.pdf')
fig.show(width=800)


In [ ]:
def add_votemap(fig, depth, cmapstring):
    min_votes = 8
    votemap = pygmt.xyz2grd('/Users/simon/Data/SeismicTomography/votemaps_allP_20220809/SubMachine_votemap_depth_slice_{:0.0f}.txt'.format(depth),
                            region='d', spacing='0.5d')
    votemap_filt = pygmt.grdfilter(grid=votemap, distance=2, filter='g500', coltypes='g')
    votemap_filt.data[votemap_filt.data<(min_votes+0.1)] = np.nan
    pygmt.makecpt(cmap=cmapstring, series=[min_votes,20,4], no_bg=True)
    fig.grdview(grid=votemap_filt, cmap=True, transparency=70, #nan_transparent=True,
                projection='Q0/37.5/10c', region='g', perspective=perspective, 
                zsize="0.1c",
                surftype="s")


def tomotectonic_model(fig, reconstruction_model, reconstruction_time,
                       region='d', projection='Q0/37.5/10c'):

    fig.basemap(projection='Q0/37.5/10c', region='g', perspective=perspective, frame=['afg','+gwhite'])

    add_votemap(fig, 1750, 'purple,purple')
    add_votemap(fig, 1500, 'royalblue,royalblue')
    add_votemap(fig, 1250, 'orange,orange')
    add_votemap(fig, 1000, 'red,red')

    reconstruction_model.polygon_snapshot('coastlines', reconstruction_time).plot(fig, color='gray70', transparency=80, pen='1p,gray20',
                                                                                  projection='Q0/37.5/10c', region='g', perspective=perspective)
    reconstruction_model.plate_snapshot(
        reconstruction_time).plot_subduction_zones(fig, pen='1.5p,black', gap=9, size=5, color='white', transparency=40,
                                                   projection='Q0/37.5/10c', region='g', perspective=perspective)

    fig.text(x=0.11,y=0.95,text='Tomotectonic',
                 region='0/1/0/1', projection='x5c', font='14p', no_clip=True)
    fig.text(x=0.0,y=0.12,text='{:0.0f} Ma'.format(reconstruction_time),
                 region='0/1/0/1', projection='x5c', font='14p', no_clip=True)

    pygmt.makecpt(categorical=True, cmap='red,orange,royalblue,purple', color_model='+c1000,1250,1500,1750', no_bg=True)
    with pygmt.config(FONT_ANNOT_PRIMARY='9p,white', FONT_LABEL='10p', MAP_TICK_LENGTH_PRIMARY='2p', MAP_FRAME_PEN='1p,black'):
        fig.colorbar(position='JBR+jBR+o-6.0c/0.25c+w3.5c/0.33c+h', frame=['+n','xa1+lDepth of Tomography Slice'])
    with pygmt.config(FONT_LABEL='10p', MAP_TICK_LENGTH_PRIMARY='2p', MAP_FRAME_PEN='1p,black'):
        fig.colorbar(position='JBR+jBR+o-6.0c/0.25c+w3.5c/0.33c+h')


fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")
tomotectonic_model(fig, C2020, reconstruction_time=100)
fig.savefig('./figures/tomotectonics.pdf')
fig.show(width=800)



In [ ]:

fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")

with fig.subplot(nrows=2, ncols=2, figsize=("24c", "12c"), frame="lrtb"):
    with fig.set_panel(panel=0):  # sets the current panel
        continental_drift(fig, M2021, reconstruction_time=600, projection='Q120/37.5/10c')

    with fig.set_panel(panel=1):
        topological_model(fig, M2016, reconstruction_time=330)

    with fig.set_panel(panel=2):
        tomotectonic_model(fig, C2020, reconstruction_time=100)
        
    with fig.set_panel(panel=3):
        deforming_model(fig, M2019, reconstruction_time=35)

fig.show(width=1200)
        